# Text Translation and Sentiment Analysis using Transformer Training and Optimization

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_metric, load_dataset, Dataset
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline, TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [21]:
import os

# NOTE: You may need to add two CA cert files to the CA environment variable: one for HuggingFace, and one for raw.githubusercontent 
# The requests library seems to have a bug where it doesn't recognize environment variables with more than one path separated by the standard convention of the ";" delimeter and no spaces

path1 = './certs/<PATH-to-HuggingFace.crt>' # certificate for HuggingFace (translation transformer)
path2 = './certs/<PATH-to-raw.githubusercontent.crt>' # certificate for raw.githubusercontent.com/huggingface/datasets (training)

bundlePath = os.pathsep.join([path1, path2]) # same as ";".join

os.environ['REQUESTS_CA_BUNDLE'] = path2

#print(os.getenv('REQUESTS_CA_BUNDLE'))

In [3]:
torchVersion = torch.__version__
print(f'PyTorch Version: {torchVersion}')
print("CUDA support PyTorch version detected") if "cu" in torchVersion else None
print("\n")

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    torch.cuda.empty_cache()
    print(f'Memory allocated: {torch.cuda.memory_allocated()}')
    print(f'Max memory allocated: {torch.cuda.max_memory_allocated()}')

PyTorch Version: 2.3.0+cu118
CUDA support PyTorch version detected


CUDA is available!  Training on GPU ...
Memory allocated: 0
Max memory allocated: 0


### Get data from `.csv` files and then preprocess data

In [4]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> tuple[pd.DataFrame, list[int]]:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")

    df_fr.columns = df_eng.columns
    df_sp.columns = df_eng.columns

    dfs = [df_eng, df_fr, df_sp]
    df_lengths = [len(df) for df in dfs]
    source_languages = ["English", "French", "Spanish"]

    df_combined = pd.concat(dfs, axis=0, ignore_index=True)

    source_columns = [source_languages[i] for i, length in enumerate(df_lengths) for _ in range(length)]
    # source_columns = []
    # for i, length in enumerate(df_lengths):
    #     source_columns.extend([source_languages[i]] * length)

    df_combined["Original Language"] = source_columns

    return df_combined, df_lengths

df, df_lengths = preprocess_data()

In [5]:
print(df)

                      Movie / TV Series         Year  \
0             The Shawshank Redemption          1994   
1                      The Dark Knight          2008   
2                          Forrest Gump         1994   
3                        The Godfather          1972   
4                             Inception         2010   
5                     Blade Runner 2049         2017   
6           Scott Pilgrim vs. the World         2010   
7                         The Nice Guys         2016   
8               Solo: A Star Wars Story         2018   
9                            The Island         2005   
10                           La La Land         2016   
11                         Intouchables         2011   
12                               Amélie         2001   
13                        Les Choristes         2004   
14  Le Fabuleux Destin d'Amélie Poulain         2001   
15                     Le Dîner de Cons         1998   
16       La Tour Montparnasse Infernale         

### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [ ]:
# load translation models and tokenizers
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return decoded

In [7]:
# filter reviews in French and translate to English
fr_reviews = df[df['Original Language'] == "French"]["Review"]
fr_reviews_en = [translate(rev, fr_en_model, fr_en_tokenizer) for rev in fr_reviews.tolist()]

# filter synopsis in French and translate to English
fr_synopsis = df[df['Original Language'] == "French"]["Synopsis"]
fr_synopsis_en = [translate(synop, fr_en_model, fr_en_tokenizer) for synop in fr_synopsis.tolist()]

# filter reviews in Spanish and translate to English
es_reviews = df[df['Original Language'] == "Spanish"]["Review"]
es_reviews_en = [translate(rev, es_en_model, es_en_tokenizer) for rev in es_reviews.tolist()]

# filter synopsis in Spanish and translate to English
es_synopsis = df[df['Original Language'] == "Spanish"]["Synopsis"]
es_synopsis_en = [translate(synop, es_en_model, es_en_tokenizer) for synop in es_synopsis.tolist()]

In [9]:
# get original english reviews and synopses
en_reviews = df[df['Original Language'] == "English"]["Review"]
en_synopses = df[df['Original Language'] == "English"]["Synopsis"]

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

translated_reviews = [fr_reviews_en[j] if i == 1 else es_reviews_en[j] if i == 2 else en_reviews[j] for i, length in enumerate(df_lengths) for j in range(length)]
df['Review'] = translated_reviews

translated_synopses = [fr_synopsis_en[j] if i == 1 else es_synopsis_en[j] if i == 2 else en_synopses[j] for i, length in enumerate(df_lengths) for j in range(length)]
df['Synopsis'] = translated_synopses

df.sample(30)

,Movie / TV Series,Year,Synopsis,Review,Original Language
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",French
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",French
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [10]:
# load sentiment analysis model
model_name = "distilbert-base-uncased" #"bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def analyze_sentiment(text, classifier, tokenizer, usePipe=False):
    """
    function to perform sentiment analysis on a text using a model
    """

    tokens = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
    outputs = classifier(**tokens)
    predicted_class = torch.argmax(outputs.logits, dim=1)
    sentiment = "positive" if predicted_class.item() == 1 else "negative"

    # Use Sentiment analysis pipeline from transformers library
    if usePipe:
        # Initialize the sentiment analysis pipeline
        model = pipeline("sentiment-analysis")
        output = model(text)
        sentiment = output[0]['label']


    return sentiment

In [12]:
# perform sentiment analysis on reviews and store results in new column
sentiments = []
for review in df["Review"]:
    sentiment = analyze_sentiment(review, model, tokenizer, usePipe=True)
    sentiments.append(sentiment)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\justhess\OneDrive - Agilent Technologies\Documents\Python\ML stuff\Udacity projects\proj3\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dist

In [13]:
df['Sentiment'] = sentiments

df.sample(30)

,Movie / TV Series,Year,Synopsis,Review,Original Language,Sentiment
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English,NEGATIVE
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish,POSITIVE
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",Spanish,POSITIVE
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French,POSITIVE
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",French,NEGATIVE
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish,NEGATIVE
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Spanish,NEGATIVE
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English,POSITIVE
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",French,POSITIVE
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French,POSITIVE


In [14]:
# export the results to a .csv file
df.to_csv("reviews_with_sentiment.csv", index=False)

### Split data into training and testing sets

We need to convert the test and train datasets from pandas dataframes (type not modified from train_test_split) to the datasets library's Dataset object. We have to manually create the column name 'label' and 'text' for the tokenizer to use the text value by indexing by its key, and the transformer library trainer.train() method needs to have these column names specified or a ValueError will be thrown: 
```python
trainer = Trainer(
    model=model,
    ...
    train_dataset= # <tokenized-train-data-with-label-and-text-column-headers>
    eval_dataset= # <tokenized-train-data-with-label-and-text-column-headers>
    ...
)
```

In [15]:
labels = df['Sentiment']
features = df['Synopsis']

train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

# NOTE: Need to use "label" and "text" columns in train and test datasets that are input to trainer.train() method or a ValueError will be thrown:
# **ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.**
train_df = pd.DataFrame({'label': train_labels, 'text': train_features})
test_df = pd.DataFrame({'label': test_labels, 'text': test_features})

label_mapping = {"NEGATIVE": 0, "POSITIVE": 1}

# Convert labels in train_df
train_df['label'] = train_df['label'].map(label_mapping)

# Convert labels in test_df
test_df['label'] = test_df['label'].map(label_mapping)

# Convert from pandas dataframe to datasets library's Dataset object
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [16]:
def tokenize(batch):
   return tokenizer(batch["text"], padding="max_length", truncation=True)
 
# To just get the tokens, index by the ["input_ids"] key in the output of the tokenize function, and convert map output to a list
# The datasets library .map() function automatically converts to a dictionary for input into trainer.train()
tokenized_train = train_dataset.map(tokenize, batched=True)
tokenized_test = test_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 8/8 [00:00<00:00, 1448.25 examples/s]


In [24]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy", trust_remote_code=True)
   load_f1 = load_metric("f1", trust_remote_code=True)
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   #return metric.compute(predictions=predictions, references=labels)
   return {"accuracy": accuracy, "f1": f1}

To speed up training, let's use a data_collator to convert your training samples to PyTorch tensors and concatenate them with the correct amount of padding:

In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
training_args = TrainingArguments(
    output_dir="./test_trainer",
    learning_rate=2e-5,
    num_train_epochs=10,
    per_device_train_batch_size=2,  # Set desired batch size here
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    eval_accumulation_steps=1,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

# Train and evaluate the transformer on the tokenized train and test data
try:

   trainer.train()
   trainer.evaluate()
   
except RuntimeError as e:
   if 'CUDA out of memory.' in str(e):
        print("CUDA out of memory error occured")
        torch.cuda.empty_cache() 

c:\Users\justhess\OneDrive - Agilent Technologies\Documents\Python\ML stuff\Udacity projects\proj3\venv\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



















                                                


                                    

 10%|█         | 11/110 [02:06<00:30,  3.27it/s]






{'eval_loss': 0.7499402761459351, 'eval_accuracy': 0.375, 'eval_f1': 0.2857142857142857, 'eval_runtime': 0.9272, 'eval_samples_per_second': 8.628, 'eval_steps_per_second': 4.314, 'epoch': 1.0}




















                                                


                                    

 10%|█         | 11/110 [02:10<00:30,  3.27it/s]






{'eval_loss': 0.7501955628395081, 'eval_accuracy': 0.375, 'eval_f1': 0.2857142857142857, 'eval_runtime': 0.9743, 'eval_samples_per_second': 8.211, 'eval_steps_per_second': 4.106, 'epoch': 2.0}




















                                                


                                    

 10%|█         | 11/110 [02:14<00:30,  3.27it/s]






...



















                                                


                                    

 10%|█         | 11/110 [02:39<00:30,  3.27it/s]






{'eval_loss': 0.7369903326034546, 'eval_accuracy': 0.375, 'eval_f1': 0.2857142857142857, 'eval_runtime': 0.8922, 'eval_samples_per_second': 8.966, 'eval_steps_per_second': 4.483, 'epoch': 9.0}




















                                                


                                    

 10%|█         | 11/110 [02:44<00:30,  3.27it/s] 





                                                

100%|██████████| 110/110 [00:42<00:00,  2.59it/s]


{'eval_loss': 0.767660915851593, 'eval_accuracy': 0.375, 'eval_f1': 0.2857142857142857, 'eval_runtime': 0.9747, 'eval_samples_per_second': 8.208, 'eval_steps_per_second': 4.104, 'epoch': 10.0}
{'train_runtime': 42.5351, 'train_samples_per_second': 5.172, 'train_steps_per_second': 2.586, 'train_loss': 0.5402238325639205, 'epoch': 10.0}




100%|██████████| 4/4 [00:00<00:00,  6.21it/s]
